In [ ]:
import datetime as dt
import numpy as np
import numba as nb
import pandas as pd
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from gas_storage.gas_price_simulations import GasPriceSimulations

### Sets and parameters
$$
\begin{align*}
T &\dots \text{number of days} \\
N &\dots \text{number of simulations} \\
I_{max} &\dots \text{maximal inventory state} \\
q_{in},\;q_{out} &\dots \text{maximal injection and withdrawal rate} \\
\mathcal{I} &= \{0,\Delta I, 2\Delta I, \dots, I_{max}\} \\
t &\in \{1,\dots,T\} \\
k &\in \{1,\dots,N\} \\
S^{(k)}_t\in\mathbb{R} &\dots \text{gas spot price simulation $k$ at time $t$} \\
a &\dots \text{action on the inventory} \\
\mathcal{A}(I) &= \{a\in\{q_{in},0,q_{out}\}\mid I+a\in\mathcal{I} \}\\
\end{align*}
$$

### Variables
Inventory state (realized) at time $t$
$$
\begin{align*}
I_t &\in \mathcal{I} \\
\end{align*}
$$

Decision variable
$$
\begin{align*}
a_t &\in \mathcal{A}(I_t)
\end{align*}
$$

##### Value function
$$
\begin{align*}
V_t(I) &= \sup_{\{a_{\tau}\}_{\tau=t}^{T-1}}\mathbb{E}[\sum^{T-1}_{\tau=t}(-a_{\tau}S_{\tau})+I_T S_T\mid I_t=I] \\
V_t(I) &= \sup_{\{a_{\tau}\}_{\tau=t}^{T-1}}\mathbb{E}[\sum^{T-1}_{\tau=t}(-a_{\tau}S_{\tau})+(I+\sum^{T-1}_{\tau=t}a_{\tau})S_T] \\
\end{align*}
$$

##### Bellman equation
$$
V_t(I) = \max_{a\in \mathcal{A}(I)}\{-a S_t + \mathbb{E}[V_{t+1}(I+a)\mid S_t]\} \\
$$
Value today $=$ immediate cashflow from the best chosen action $+$ expected value of tomorrow (expected continuation value). <br>
This expected value depends on today's price - the next day action $a_t$ depends on the price and inventory.

##### Constraints
$$
\begin{align*}
I_{t+1} &= I_t + a_t \\
V_T(I) &= I\cdot S_T,\quad \forall I\in\mathcal{I} \\
\end{align*}
$$

##### Expectation handling
$$
\mathbb{E}[V_{t+1}(I_{t+1})\mid S_t,I_t]
$$
What is the average future value of the storage tomorrow, given what I know today: today's price $S_t$ and today's inventory $I_t$.

##### Summary
At each time $t$ we compute the value of each possible inventory state using many simulated prices to estimate the expected future value.

### Value function example
$$
\begin{align*}
t &\dots \text{today} \\
S_t &= 30\; \text{EUR/MWh} \\
I_t &= 50\; \text{MWh} \\
a_t &\in \{-5, 0, 5\} \\
V_t(I_t) &= \text{Maximum expected future profit from today onward} \\
\end{align*}
$$

##### Action 1 - inject ($a_t = 5$)
$$
\begin{align*}
I_{t+1} &= 50+5=55 \\
-a_t\cdot S_t &= -5\cdot 30 = -150 \\
V_{t+1}(55) &= \text{?} \\
\text{value} &: −150+\mathbb{E}[V_{t+1}​(55) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Action 2 - hold ($a_t = 0$)
$$
\begin{align*}
I_{t+1} &= 50 \\
-a_t\cdot S_t &= 0 \\
\text{value} &: \mathbb{E}[V_{t+1}​(50) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Action 3 - withdraw ($a_t = -5$)
$$
\begin{align*}
I_{t+1} &= 45 \\
-a_t\cdot S_t &= -(-5)\cdot 30 = 150 \\
\text{value} &: 150+\mathbb{E}[V_{t+1}​(45) \mid S_t​=30,I_t​=50]
\end{align*}
$$

##### Decision
$$
\begin{align*}
\text{inject} &: −150+\mathbb{E}[V_{t+1}​(55) \mid S_t​=30,I_t​=50] \\
\text{hold} &: \mathbb{E}[V_{t+1}​(50) \mid S_t​=30,I_t​=50] \\
\text{withdraw} &: 150+\mathbb{E}[V_{t+1}​(45) \mid S_t​=30,I_t​=50] \\
\end{align*}
$$

### Specific example

$$
\begin{align*}
T &= 3 \\
t &\in \{1,2,3\} \\
\mathcal{I} &= \{0, 1\} \\
\end{align*}
$$

That means: $$\mathcal{A}(I) = \{a\in\{-1, 0, 1\}\mid I+a\in\mathcal{I}\}$$

Particularly:
$$
\begin{align*}
I = 0 &\dots \mathcal{A}(I)=\{0, 1\} \\
I = 1 &\dots \mathcal{A}(I)=\{-1, 0\} \\
\end{align*}
$$

Simulations
$$
\begin{align*}
N &= 2 \\
k &\in \{1, 2\} \\
S^{(1)} &= (10, 12, 15) \\
S^{(2)} &= (10, 8, 5) \\
\end{align*}
$$

Payoff at time $t$: $$CF_t = -a_t S_t$$

Terminal payoff: $$V_3(I)=I\cdot S_3$$

#### Backward induction
$t=3,\quad I=0$
$$V_3(0) = 0 $$

$t=3,\quad I=1$
$$V_3(1) = S_3$$



$t=2,\quad I=0,\quad a=0$
$$V_2(0) = 0 + \mathbb{E}[V_3(0+0)] = 0 $$
$t=2,\quad I=0,\quad a=1$
$$V_2(0) = -S_2 + \mathbb{E}[V_3(0+1)] = -S_2 + \mathbb{E}[S_3] = -S_2 + 10 $$
Total:
$$V_2(0) = \max\{0, -S_2+10\}$$

$t=2,\quad I=1,\quad a=0$
$$V_2(1) = 0 + \mathbb{E}[V_3(1)] = \mathbb{E}[S_3] = 10 $$
$t=2,\quad I=1,\quad a=-1$
$$V_2(1) = S_2 + \mathbb{E}[V_3(1-1)] = S_2 + 0 = S_2 $$
Total:
$$V_2(1) = \max\{10, S_2\}$$



$t=1,\quad I=0,\quad a=0$
$$V_1(0) = 0 + \mathbb{E}[V_2(0+0)] = \mathbb{E}[\max\{0, -S_2+10\}] = \dfrac{1}{2}(0+2) = 1 $$
$t=1,\quad I=0,\quad a=1$
$$V_1(0) = -S_1 + \mathbb{E}[V_2(0+1)] = -S_1 + \mathbb{E}[\max\{10, S_2\}] = -S_1 + \dfrac{1}{2}(12+10) = 11-S_1 $$
Total:
$$V_1(0) = \max\{1, 11-S_1\}$$

$t=1,\quad I=1,\quad a=0$
$$V_1(1) = 0 + \mathbb{E}[V_2(1+0)] = \mathbb{E}[\max\{10, S_2\}] = 11 $$
$t=1,\quad I=1,\quad a=-1$
$$V_1(1) = S_1 + \mathbb{E}[V_2(1-1)] = S_1 + \mathbb{E}[\max\{0, -S_2+10\}] = S_1 + 1$$
Total:
$$V_1(1) = \max\{11,S_1+1\}$$

##### Summary
Now we can define initial state of inventory: at $t=1$ we set $I=0$. Then the storage value at is $V_1(0)=\max\{1, 11-S_1\}$. If it holds that $\mathbb{E}[S_1]=10$, then the value is $\mathbb{E}[V_1(0)]=1$.

Or we can set the inventory to be full at the beginning. Then $\mathbb{E}[V_1(1)]=11$.

##### Takeaway
In real problems the number of paths is higher, distributions of prices are more granular, more time steps. LSMC then steps in when calculating $$\mathbb{E}[V_{t+1}(I+a)\mid S_t],$$
which can't be calculated analytically.


### Adding LSMC

We can compute:
$$Y^{(n)} = V^{(n)}_3(I)$$

Then regress:
$$Y^{(n)} = \beta_0 + \beta_1S_2^{(n)} + \beta_2(S_2^{(n)})^2$$

With this regression we can estimate:
$$\mathbb{E}[V_3(I)\mid S_2]$$

In [ ]:
N = 100000
mu, rho, sigma = 10, 0.4, 2
eps2 = np.random.randn(N)
eps3 = np.random.randn(N)

# simulate prices
S1 = mu + sigma*np.random.randn(N)
S2 = mu + rho*(S1-mu) + sigma*eps2
S3 = mu + rho*(S2-mu) + sigma*eps3

# t=3 (terminal)
V3_0 = np.zeros(N)
V3_1 = S3

# t=2: regress V3 on S2, DECIDE with fitted, RECORD with actual
X = np.column_stack([np.ones(N), S2, S2**2])
beta_0 = np.linalg.lstsq(X, V3_0, rcond=None)[0]
beta_1 = np.linalg.lstsq(X, V3_1, rcond=None)[0]

EV3_0 = X @ beta_0   # fitted E[V3(0)|S2] ≈ 0
EV3_1 = X @ beta_1   # fitted E[V3(1)|S2]

# I=0: decide a=0 vs a=1 using fitted (EV); record actual (V)
# coming from: V2_0 = np.maximum(0, EV3_1 - S2)
V2_0 = np.where(-S2 + EV3_1 > EV3_0, -S2 + V3_1, V3_0)
# I=1: decide a=0 vs a=-1 using fitted; record actual
# coming from: V2_1 = np.maximum(EV3_1, S2)
V2_1 = np.where(EV3_1 > S2 + EV3_0, V3_1, S2 + V3_0)

# t=1: regress V2 on S1, DECIDE with fitted, RECORD with actual
X = np.column_stack([np.ones(N), S1, S1**2])
beta_0 = np.linalg.lstsq(X, V2_0, rcond=None)[0]
beta_1 = np.linalg.lstsq(X, V2_1, rcond=None)[0]

EV2_0 = X @ beta_0   # fitted E[V2(0)|S1]
EV2_1 = X @ beta_1   # fitted E[V2(1)|S1]

# I=0: decide a=0 vs a=1 using fitted; record actual
V1_0 = np.where(-S1 + EV2_1 > EV2_0, -S1 + V2_1, V2_0)
# I=1: decide a=0 vs a=-1 using fitted; record actual
V1_1 = np.where(EV2_1 > S1 + EV2_0, V2_1, S1 + V2_0)

display(np.mean(V1_0))
display(np.mean(V1_1))